In [80]:
import numpy as np                       # Import necessary libraries
import pandas as pd                      # library for numerical computing in Python
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [110]:
df = pd.read_excel("Random_leak_data_by_python.xlsx")
df.head()

,Timestamp,Pressure,Temperature,Frequency,Amplitude,Latitude,Longitude,Leak Condition
0,2024-03-19 23:57:09.580,22.220362,17.555440,18095.837476,1.002457,23.736042,90.520816,1
1,2024-03-19 23:58:09.580,38.178473,16.504456,17337.665986,1.022310,23.643534,90.898804,0
2,2024-03-19 23:59:09.580,40.636551,15.167562,12844.363662,1.016013,23.045692,90.829612,0
3,2024-03-20 00:00:09.580,32.108980,7.294893,12493.664206,1.117639,23.429280,91.554377,1
4,2024-03-20 00:01:09.580,37.839198,17.073648,18637.472310,0.994379,23.367979,90.680205,0


In [3]:
df.corr()['Leak Condition']

,Leak Condition
Timestamp,-0.055999
Pressure,-0.264892
Temperature,-0.366289
Frequency,0.149647
Amplitude,0.330586
Latitude,0.059598
Longitude,0.020988
Leak Condition,1.000000


The correlation of input data and Leak condition
when a leak happens, pressure and temperature drops
Frequency and Amplitude increases at leak point.

In [5]:
df.describe()

,Timestamp,Pressure,Temperature,Frequency,Amplitude,Latitude,Longitude,Leak Condition
count,1000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,2024-03-20 08:16:39.579999744,32.631475,14.709637,17120.234660,1.002017,23.066882,91.119195,0.24900
min,2024-03-19 23:57:09.580000,20.029966,5.002373,12014.294483,0.800259,22.342553,90.405319,0.00000
25%,2024-03-20 04:06:54.580000,25.828251,9.645153,14757.183517,0.902935,22.733037,90.762046,0.00000
50%,2024-03-20 08:16:39.580000,32.584079,14.653433,17208.472453,0.999760,23.063859,91.109336,0.00000
75%,2024-03-20 12:26:24.580000,39.195774,19.633735,19588.242969,1.102487,23.418566,91.480259,0.00000
max,2024-03-20 16:36:09.580000,44.982209,24.969157,21994.159679,1.199526,23.774315,91.814591,1.00000
std,NaN,7.453580,5.823016,2903.636426,0.113768,0.407325,0.412967,0.43265


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Timestamp       1000 non-null   datetime64[ns]
 1   Pressure        1000 non-null   float64       
 2   Temperature     1000 non-null   float64       
 3   Frequency       1000 non-null   float64       
 4   Amplitude       1000 non-null   float64       
 5   Latitude        1000 non-null   float64       
 6   Longitude       1000 non-null   float64       
 7   Leak Condition  1000 non-null   int64         
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 62.6 KB


In [85]:
# Split the data into features (X) and target variable (y)

X = df.drop(columns=['Timestamp', 'Latitude', 'Longitude', 'Leak Condition'])
y = df['Leak Condition']

In [111]:
# Split the data into features (X) and target variable (y)

X = df.drop(columns=['Timestamp', 'Leak Condition'])
y = df['Leak Condition']

In [65]:
X.head()

,Pressure,Temperature,Frequency,Amplitude
0,22.220362,17.555440,18095.837476,1.002457
1,38.178473,16.504456,17337.665986,1.022310
2,40.636551,15.167562,12844.363662,1.016013
3,32.108980,7.294893,12493.664206,1.117639
4,37.839198,17.073648,18637.472310,0.994379


In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Importing train_test_split function from sklearn library
# Splitting the dataset into training and testing sets with 80% for training and 20% for testing
# random_state=1 ~ The data split remains the same across runs.

In [11]:
X_train.shape

(800, 4)

In [113]:
from sklearn.preprocessing import MinMaxScaler                    # Importing MinMaxScaler from sklearn.preprocessing
scaler = MinMaxScaler()                                           # Initializing MinMaxScaler object
X_train_scaled = scaler.fit_transform(X_train)                    # Scaling the training features using Min-Max scaling
X_test_scaled = scaler.fit_transform(X_test)                      # Scaling the testing features using Min-Max scaling

In [13]:
X_train_scaled

array([[0.6420244 , 0.57587339, 0.83486527, 0.99978854],
       [0.70073539, 0.76830139, 0.97713379, 0.3537076 ],
       [0.51063136, 0.99588486, 0.04040676, 0.23433096],
       ...,
       [0.91038818, 0.35685424, 0.77662268, 0.67575959],
       [0.70360228, 0.85030101, 0.84801772, 0.03193649],
       [0.76139835, 0.96737265, 0.08222579, 0.98922814]])

In [114]:
from keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# The Dense layer represents a fully connected layer in a neural network,
    # where each neuron in the layer is connected to every neuron in the preceding layer.
# The Sequential class is used to build neural network models layer by layer, in a sequential manner.
# Flatten layer is another type of layer used in neural networks, which is used to flatten the input data into a one-dimensional array.

In [20]:
model  = Sequential()
model.add(Dense(8, activation = 'relu', input_dim = 4))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1,activation ='sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [21]:
history = model.fit(X_train_scaled, y_train, epochs = 100, batch_size= 32, validation_data=(X_test_scaled, y_test))

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7901 - loss: 0.6563 - val_accuracy: 0.7950 - val_loss: 0.6251
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7805 - loss: 0.6182 - val_accuracy: 0.7950 - val_loss: 0.5836
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7740 - loss: 0.5763 - val_accuracy: 0.7850 - val_loss: 0.5390
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7701 - loss: 0.5417 - val_accuracy: 0.7850 - val_loss: 0.5006
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7694 - loss: 0.5059 - val_accuracy: 0.7900 - val_loss: 0.4690
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7380 - loss: 0.5064 - val_accuracy: 0.8050 - val_loss: 0.4453
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7707 - loss: 0.4707 - val_accuracy: 0.8200 - val_loss: 0.4250
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8074 - loss: 0.4334 - val_accuracy: 0.8350 - 

In [23]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [24]:
import kerastuner as kt

<ipython-input-24-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [28]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step = 8),
                      activation = hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']), input_dim= 4))
    else:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step = 8),
                      activation = hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']), input_dim= 4))
    counter +=1
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam', 'adadelta','sgd', 'nadam']),
                loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [29]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'Final')

Reloading Tuner from mydir/Final/tuner0.json


In [30]:
tuner.search(X_train_scaled, y_train, epochs = 10, validation_data=(X_test_scaled, y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.6100000143051147

Best val_accuracy So Far: 0.6100000143051147
Total elapsed time: 00h 01m 09s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 88,
 'activation0': 'tanh',
 'optimizer': 'adadelta',
 'units1': 88,
 'activation1': 'tanh',
 'units2': 48,
 'activation2': 'sigmoid',
 'units3': 104,
 'activation3': 'relu'}

In [32]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adadelta', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [33]:
history1 = model.fit(X_train_scaled, y_train, batch_size =32, epochs = 200, validation_data=(X_test_scaled, y_test))

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6321 - loss: 0.6706 - val_accuracy: 0.6100 - val_loss: 0.6708
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6434 - loss: 0.6694 - val_accuracy: 0.6150 - val_loss: 0.6703
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6464 - loss: 0.6700 - val_accuracy: 0.6200 - val_loss: 0.6699
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6750 - loss: 0.6660 - val_accuracy: 0.6250 - val_loss: 0.6694
Epoch 5/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6789 - loss: 0.6636 - val_accuracy: 0.6350 - val_loss: 0.6690
Epoch 6/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6910 - loss: 0.6653 - val_accuracy: 0.6350 - val_loss: 0.6685
Epoch 7/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6779 - loss: 0.6628 - val_accuracy: 0.6350 - val_loss: 0.6680
Epoch 8/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7008 - loss: 0.6634 - val_accuracy: 0.6350 - 

# Adding Dropout

In [34]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step = 8),
                      activation = hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']), input_dim= 4))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step = 8),
                      activation = hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']), input_dim= 4))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter +=1
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam', 'adadelta','sgd', 'nadam']),
                loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [35]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'Final_final')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
tuner.search(X_train_scaled, y_train, epochs = 10, validation_data=(X_test_scaled, y_test))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.7850000262260437

Best val_accuracy So Far: 0.8799999952316284
Total elapsed time: 00h 00m 19s


In [37]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 88,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1}

In [38]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [39]:
history2 = model.fit(X_train_scaled, y_train, batch_size =32, epochs = 200, validation_data=(X_test_scaled, y_test))

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8041 - loss: 0.4338 - val_accuracy: 0.8650 - val_loss: 0.3497
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8240 - loss: 0.4306 - val_accuracy: 0.8700 - val_loss: 0.3417
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8204 - loss: 0.4035 - val_accuracy: 0.8650 - val_loss: 0.3350
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8079 - loss: 0.3928 - val_accuracy: 0.8700 - val_loss: 0.3311
Epoch 5/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8178 - loss: 0.4289 - val_accuracy: 0.8750 - val_loss: 0.3301
Epoch 6/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8271 - loss: 0.3764 - val_accuracy: 0.8700 - val_loss: 0.3282
Epoch 7/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8412 - loss: 0.3806 - val_accuracy: 0.8550 - val_loss: 0.3295
Epoch 8/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8275 - loss: 0.3821 - val_accuracy: 0.8500 -

In [40]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128),
                      activation = hp.Choice('activation' + str(i), values=['relu', 'tanh']), input_dim= 4))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128),
                      activation = hp.Choice('activation' + str(i), values=['relu', 'tanh']), input_dim= 4))
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter +=1
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam', 'adadelta','sgd', 'nadam']),
                loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [41]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'Final_final')

Reloading Tuner from mydir/Final_final/tuner0.json


In [42]:
tuner.search(X_train_scaled, y_train, epochs = 5, validation_data=(X_test_scaled, y_test))

In [43]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 88,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1}

In [44]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [45]:
history4 = model.fit(X_train_scaled, y_train, batch_size =32, epochs = 200, validation_data=(X_test_scaled, y_test))

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8215 - loss: 0.4000 - val_accuracy: 0.8800 - val_loss: 0.3489
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7911 - loss: 0.4382 - val_accuracy: 0.8650 - val_loss: 0.3411
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8332 - loss: 0.3804 - val_accuracy: 0.8650 - val_loss: 0.3357
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8218 - loss: 0.3978 - val_accuracy: 0.8650 - val_loss: 0.3325
Epoch 5/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8189 - loss: 0.3852 - val_accuracy: 0.8600 - val_loss: 0.3316
Epoch 6/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8121 - loss: 0.3982 - val_accuracy: 0.8350 - val_loss: 0.3365
Epoch 7/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8298 - loss: 0.3851 - val_accuracy: 0.8650 - val_loss: 0.3255
Epoch 8/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8176 - loss: 0.3951 - val_accuracy: 0.8550 - 

In [52]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight


# Compute class weights to handle any potential imbalance in the dataset
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Build model function for hyperparameter tuning
def build_model(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):  # Reduced max layers for simplicity
        if counter == 0:
            model.add(Dense(hp.Int('units_' + str(i), min_value=8, max_value=128),
                            activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh']),
                            input_dim=4))  # 4 features as input
            model.add(Dropout(hp.Float('dropout_' + str(i), min_value=0.1, max_value=0.5, step=0.1)))
        else:
            model.add(Dense(hp.Int('units_' + str(i), min_value=8, max_value=128),
                            activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh']), input_dim =4))
            model.add(Dropout(hp.Float('dropout_' + str(i), min_value=0.1, max_value=0.5, step=0.1)))
        counter += 1

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'adadelta', 'nadam', 'sgd']),
                  loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [53]:
# Hyperparameter tuner setup
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='mydir',
    project_name='Final_final'
)

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Perform hyperparameter tuning
tuner.search(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test),
             callbacks=[early_stopping], class_weight=class_weights_dict)

Reloading Tuner from mydir/Final_final/tuner0.json


In [56]:
# Get the best hyperparameters and model
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 88,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units_0': 8,
 'activation_0': 'relu',
 'dropout_0': 0.1,
 'units_1': 8,
 'activation_1': 'relu',
 'dropout_1': 0.1,
 'units_2': 8,
 'activation_2': 'relu',
 'dropout_2': 0.1}

In [58]:
best_hps = tuner.get_best_hyperparameters()[0]

In [59]:
model = build_model(best_hps)

In [61]:
history4 = model.fit(X_train_scaled, y_train, batch_size=32, epochs=200, validation_data=(X_test_scaled, y_test),
                     callbacks=[early_stopping])

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6547 - loss: 0.6885 - val_accuracy: 0.7850 - val_loss: 0.6677
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7180 - loss: 0.6623 - val_accuracy: 0.7850 - val_loss: 0.6312
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7602 - loss: 0.6313 - val_accuracy: 0.7850 - val_loss: 0.5910
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7441 - loss: 0.5936 - val_accuracy: 0.7850 - val_loss: 0.5483
Epoch 5/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7275 - loss: 0.5674 - val_accuracy: 0.7850 - val_loss: 0.5120
Epoch 6/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7171 - loss: 0.5628 - val_accuracy: 0.7850 - val_loss: 0.4796
Epoch 7/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7516 - loss: 0.5167 - val_accuracy: 0.7850 - val_loss: 0.4487
Epoch 8/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7777 - loss: 0.5010 - val_accuracy: 0.8400 - 

In [91]:
model  = Sequential()
model.add(Dense(64, activation = 'relu', input_dim = 4))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(18, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(18, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(18, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1,activation ='sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [92]:
callback = EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.00001,
    patience = 100,
    verbose = 1,
    mode = 'auto',
    baseline=None,
    restore_best_weights=False
)


In [93]:
history5 = model.fit(X_train_scaled, y_train, epochs = 100, batch_size= 32, validation_data=(X_test_scaled, y_test),callbacks=callback)

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.6086 - loss: 0.7301 - val_accuracy: 0.7900 - val_loss: 0.6211
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6871 - loss: 0.6629 - val_accuracy: 0.7900 - val_loss: 0.5796
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7452 - loss: 0.5439 - val_accuracy: 0.7900 - val_loss: 0.5472
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7567 - loss: 0.5047 - val_accuracy: 0.7900 - val_loss: 0.5254
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7901 - loss: 0.4750 - val_accuracy: 0.7900 - val_loss: 0.5066
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8136 - loss: 0.4312 - val_accuracy: 0.7900 - val_loss: 0.4983
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8354 - loss: 0.4068 - val_accuracy: 0.7900 - val_loss: 0.4967
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8347 - loss: 0.3778 - val_accuracy: 0.7900 - 

# Increasing complexities

In [94]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', min_value=2, max_value=10)):  # Increased layers
        model.add(Dense(hp.Int('units_' + str(i), min_value=32, max_value=256),  # Larger units
                        activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh'])))
        model.add(Dropout(hp.Float('dropout_' + str(i), min_value=0.1, max_value=0.5)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'nadam', 'sgd']),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [95]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'Final_final')

Reloading Tuner from mydir/Final_final/tuner0.json


In [96]:
tuner.search(X_train_scaled, y_train, epochs = 5, validation_data=(X_test_scaled, y_test))

In [97]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 88,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1}

In [98]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 4 objects could not be loaded. Example error message for object <Dense name=dense, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(4, 32), Received: value.shape=(4, 88). Target variable: <KerasVariable shape=(4, 32), dtype=float32, path=sequential/dense/kernel>

List of objects that could not be loaded:
[<Dense name=dense, built=True>, <Dense name=dense_1, built=True>, <Dense name=dense_2, built=True>, <Dense name=dense_3, built=True>]

In [99]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value=32, max_value=256, step = 8),
                      activation = hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']), input_dim= 4))
    else:
      model.add(Dense(hp.Int('units' + str(i), min_value=32, max_value=256, step = 8),
                      activation = hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']), input_dim= 4))
    counter +=1
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam', 'adadelta','sgd', 'nadam']),
                loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [100]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'Final_final')

Reloading Tuner from mydir/Final_final/tuner0.json


In [101]:
tuner.search(X_train_scaled, y_train, epochs = 5, validation_data=(X_test_scaled, y_test))
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 88,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1}

In [102]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [103]:
history = model.fit(X_train_scaled, y_train, batch_size =32, epochs = 200, validation_data=(X_test_scaled, y_test))

Epoch 1/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8123 - loss: 0.4016 - val_accuracy: 0.8767 - val_loss: 0.3504
Epoch 2/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8001 - loss: 0.3841 - val_accuracy: 0.8533 - val_loss: 0.3430
Epoch 3/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8303 - loss: 0.3804 - val_accuracy: 0.8667 - val_loss: 0.3337
Epoch 4/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8304 - loss: 0.3547 - val_accuracy: 0.8633 - val_loss: 0.3308
Epoch 5/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8100 - loss: 0.3933 - val_accuracy: 0.8600 - val_loss: 0.3293
Epoch 6/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8138 - loss: 0.3848 - val_accuracy: 0.8667 - val_loss: 0.3260
Epoch 7/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8312 - loss: 0.3650 - val_accuracy: 0.8467 - val_loss: 0.3247
Epoch 8/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8332 - loss: 0.3474 - val_accuracy: 0.8633 - 

In [120]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value=32, max_value=256, step = 16),
                      activation = hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']), input_dim= 6))
    else:
      model.add(Dense(hp.Int('units' + str(i), min_value=32, max_value=256, step = 16),
                      activation = hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']), input_dim= 6))
    counter +=1
  model.add(Dense(1, activation = 'sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam', 'adadelta','sgd', 'nadam']),
                loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [121]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name = 'Fsinal_final')

Reloading Tuner from mydir/Fsinal_final/tuner0.json


In [122]:
tuner.search(X_train_scaled, y_train, epochs = 10, validation_data=(X_test_scaled, y_test))
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 208,
 'activation0': 'relu',
 'optimizer': 'nadam',
 'units1': 80,
 'activation1': 'sigmoid',
 'units2': 64,
 'activation2': 'tanh',
 'units3': 80,
 'activation3': 'sigmoid',
 'units4': 96,
 'activation4': 'tanh',
 'units5': 208,
 'activation5': 'sigmoid',
 'units6': 176,
 'activation6': 'sigmoid',
 'units7': 208,
 'activation7': 'sigmoid'}